# CPRD Notebook:
## Evaluation of the pre-trained SurvivEHR-CR model for causal sequence modelling on a supervised cohort study.

Cohort study: predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population.

This notebook quantifies the degredation in performance (due to distributional shift) observed when applying the pre-trained model to a sub-population.

In [2]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
import wandb
from tqdm import tqdm
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.examples.modelling.SurvStreamGPT.run_experiment import run
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling
from CPRD.examples.modelling.SurvStreamGPT.setup_causal_experiment import setup_causal_experiment, CausalExperiment

import time
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import os
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000

torch.manual_seed(1337)
torch.set_float32_matmul_precision('medium')

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

 # TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28   

Using device: cuda.
env: SLURM_NTASKS_PER_NODE=28


## Load configurations
The default configuration is for pre-training. Here we modify as necesssary

Here we choose to load in the configuration for a small **pre-trained** 11.4M parameter model, named "CR_11M". We specfiy the `causal` (equivalently `pre-train` or `self-supervised`) experiment type, which will lead to running the ```CausalExperiment```. 

We tell this experiment that no further training is needed. Additionally, we do choose to perform testing (true by default). As this is a causal model, this would not test the ability to predict the outcomes of interest, but to perform the causal modelling task on the chosen dataset. In this notebook, this is chosen to be the cohort study for predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population, and we add the folder containing this dataset to the configuration. 

```Note: As this is a supervised dataset, we need to tell the DataModule that the last event observed is a target and must be stripped. This is done by passing a list of targets to the configuration, overriding the null default. This lets the DataModule know that it should process batches as supervised, even if we are not going to use those targets in this noteobok```

Finally, we set the number of workers to be appropriate for the number of CPUs available to reduce bottlenecking, and tell the experiment that we do not want to limit the number of testing batches. In addition, we specify where we want any checkpoints to be saved to avoid bloating the repository.

In [11]:
# load the configuration file, override any settings 
with initialize(version_base=None, config_path="../../confs", job_name="testing_zeroshot_notebook"):
    cfg = compose(config_name="config_CompetingRisk37M", 
                  overrides=[# Experiment setup
                             "experiment.type='causal'",
                             "experiment.run_id='notebook-test3'", # CR_11M_24_11_01_big_posencscale_
                             "experiment.train=False",
                             "experiment.test=True",
                             # Dataloader
                             "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                             "data.min_workers=12",
                             # Optimiser
                             "optim.limit_test_batches=0.01",
                            ]
                 )    
# print(OmegaConf.to_yaml(cfg))

wandb.finish()
model, dm = run(cfg)
print(f"Loaded model with {sum(p.numel() for p in model.parameters())/1e6} M parameters")
wandb.finish()

save_path = f"/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/{cfg.experiment.run_id}/"

INFO:root:Running cr on 72 CPUs and 1 GPUs
INFO:root:====================================================================================================
INFO:root:# Loading DataModule for dataset /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/. This will be loaded in causal form.
INFO:root:====================================================================================================
INFO:root:Creating unsupervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/file_row_count_dict_test.pickle
INF

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        Test:Cinter        │    0.9866194395177743     │
│      Test:Cintra100       │    0.7414448669201521     │
│      Test:Cintra101       │    0.9182509505703422     │
│      Test:Cintra104       │    0.7946768060836502     │
│      Test:Cintra106       │    0.7675176534492125     │
│      Test:Cintra107       │    0.6376425855513307     │
│       Test:Cintra11       │    0.34980988593155893    │
│      Test:Cintra112       │     0.667300380228137     │
│      Test:Cintra113       │    0.8517110266159695     │
│      Test:Cintra114       │    0.8920152091254753     │
│      Test:Cintra118       │    0.9602488765986863     │
│      Test:Cintra119       │    0.9546768060836502     │
│      Test:Cintra120       │    0.7072243346007605     │
│      Test:Cintra123       │    0.9693282636248415     │
│      Test:Cintra124       │    0.8783269961977186     │
│      Test:Cintra126       │    0.9357068786726582     │
│      Test:Cintra127       │    0.8935361216730038     │
│      Test:Cintra129       │    0.9225285171102662     │
│      Test:Cintra130       │    0.9761301225179554     │
│      Test:Cintra131       │      0.9709336713139      │
│      Test:Cintra133       │    0.9166954718285517     │
│      Test:Cintra134       │    0.9093789607097591     │
│      Test:Cintra135       │     0.923954372623574     │
│      Test:Cintra136       │     0.939445148570624     │
│      Test:Cintra141       │    0.7934093789607098     │
│      Test:Cintra143       │     0.895226024503591     │
│      Test:Cintra144       │    0.9797211660329531     │
│      Test:Cintra145       │    0.8598587724063009     │
│      Test:Cintra146       │    0.9074635792711581     │
│      Test:Cintra148       │    0.8824217607487571     │
│      Test:Cintra149       │    0.9368821292775665     │
│      Test:Cintra150       │    0.9534220532319393     │
│      Test:Cintra152       │    0.9361759913090714     │
│      Test:Cintra153       │    0.9932403886776515     │
│      Test:Cintra154       │    0.9286016054076887     │
│      Test:Cintra155       │     0.982074959261271     │
│      Test:Cintra156       │    0.9819804926434127     │
│      Test:Cintra157       │    0.9839903942365421     │
│      Test:Cintra158       │    0.9441297431520155     │
│      Test:Cintra159       │    0.9863117870722432     │
│      Test:Cintra160       │    0.9489407930472569     │
│      Test:Cintra161       │    0.9495197118270958     │
│      Test:Cintra162       │    0.9416983523447402     │
│      Test:Cintra163       │    0.9619771863117871     │
│      Test:Cintra164       │    0.9918884664131816     │
│      Test:Cintra165       │    0.9857414448669202     │
│      Test:Cintra166       │    0.9864204236827812     │
│      Test:Cintra167       │    0.9547290874524715     │
│      Test:Cintra168       │    0.9702039405461464     │
│      Test:Cintra169       │    0.9628221377270807     │
│      Test:Cintra170       │    0.9818086930589736     │
│      Test:Cintra171       │    0.9378357173033983     │
│      Test:Cintra172       │    0.9246035426133731     │
│      Test:Cintra173       │    0.9939963978387034     │
│      Test:Cintra174       │    0.9936628643852979     │
│      Test:Cintra175       │    0.9881706801858895     │
│      Test:Cintra176       │    0.9685447632215692     │
│      Test:Cintra177       │    0.9837045084193375     │
│      Test:Cintra178       │    0.9401911417120541     │
│      Test:Cintra179       │    0.9839670468948039     │
│      Test:Cintra180       │    0.9904942965779469     │
│      Test:Cintra181       │    0.9902830587241235     │
│      Test:Cintra182       │    0.9758974028484888     │
│      Test:Cintra183       │    0.9981547752180724     │
│      Test:Cintra184       │    0.9769410033116646     │
│      Test:Cintra185       │    0.8257287705956907     

Loaded model with 129.356302 M parameters


Test:Cinter,▁
Test:Cintra100,▁
Test:Cintra101,▁
Test:Cintra104,▁
Test:Cintra106,▁
Test:Cintra107,▁
Test:Cintra11,▁
Test:Cintra112,▁
Test:Cintra113,▁
Test:Cintra114,▁
Test:Cintra118,▁


# Run experiment

In [4]:
import wandb


In [7]:
dm.encode(['Diastolic_blood_pressure_5.png'])
display(dm.decode([11, 141,38]).split(" "))

['SYSTEMIC_LUPUS_ERYTHEMATOSUS',
 'Serum_total_cholesterol_level_98',
 'VISUAL_IMPAIRMENT']

In [6]:
dm.tokenizer._event_counts["EVENT"][-5:].to_list()

['NSAIDS_oral_OPTIMAL_final',
 'Diastolic_blood_pressure_5',
 'Systolic_blood_pressure_4',
 'Statins',
 'Lipid_lowering_drugs_Optimal']

# Below is deprecated. Will consolodate it when creating any distribution shift plots

# Load Pre-Trained model

In [7]:
ckpt_path = cfg.experiment.log_dir + f'checkpoints/{cfg.experiment.run_id}.ckpt'
model = SurvivalExperiment.load_from_checkpoint(ckpt_path)

NameError: name 'SurvivalExperiment' is not defined

# Initialise fine-tuning data module

In [ ]:
# Update dataset path to point to the new dataset 
cfg.data.path_to_ds = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"

# Build 
dm = FoundationalDataModule(path_to_db=cfg.data.path_to_db,
                            path_to_ds=cfg.data.path_to_ds,
                            load=True,
                            tokenizer="tabular",
                            batch_size=cfg.data.batch_size,
                            max_seq_length=cfg.transformer.block_size,
                            freq_threshold=cfg.data.unk_freq_threshold,
                            min_workers=cfg.data.min_workers,
                            overwrite_meta_information=cfg.data.meta_information_path,
                           )

vocab_size = dm.train_set.tokenizer.vocab_size
print(f"{vocab_size} vocab elements")

# list of univariate measurements to model with Normal distribution
# Extract the measurements, using the fact that the diagnoses are all up upper case.
measurements_for_univariate_regression = [record for record in dm.tokenizer._event_counts["EVENT"] if record.upper() != record]
cfg.head.tokens_for_univariate_regression = dm.encode(measurements_for_univariate_regression) 
# display(measurements_for_univariate_regression)

In [ ]:
# import pickle as pkl
# import pathlib

pkl_file_to_amend = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_test.pickle"

with open(pkl_file_to_amend, 'rb') as pickle_file:
    content = pickle.load(pickle_file)
display(content)

# new_dictionary = {}
# for key in content.keys():
#     str_to_remove = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/split=val/"
#     new_key = str(key)[len(str_to_remove):]
#     new_dictionary[new_key] = content[key]
# display(new_dictionary)


# with open(pkl_file_to_amend, 'wb') as handle:
#     pickle.dump(new_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# new_dictionary

In [ ]:
start = time.time()   # starting time
for batch in dm.train_dataloader():
    break
print(f"batch loaded in {time.time()-start} seconds")    
    
# for key in batch.keys():
#     print(f"{key}".ljust(20) + f"{batch[key].shape}")

# tokens = batch["tokens"][0].tolist()    
# sentence = dm.decode(tokens).split(" ")
# for token, value in zip(sentence, batch["values"][0].tolist()):
#     print(f"{token}:".ljust(40) + f"{value}")

In [ ]:
display(batch.keys())
display(convert_batch_to_none_causal(batch).keys())

print(batch["static_covariates"].shape)

print(dm.train_set.static_1hot)
print(dm.train_set.static_1hot["SEX"].categories_)
print(dm.train_set.static_1hot["IMD"].categories_)
print(dm.train_set.static_1hot["ETHNICITY"].categories_)

## View an example sample

In [ ]:
dm.test_set.view_sample(1354, max_dynamic_events=None, report_time=True)

# Custom wrapper prediction last token

To begin with, I will just loop over samples individually to test the zero-shot capacity of SurvivEHR. 

In [ ]:


# Verifying on datamodule 
for _idx, batch in enumerate(dm.test_dataloader()):
    if _idx > 10:
        break
    print(_idx)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))
    batch = replace_last_non_pad_with_pad(batch)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))

In [ ]:
outcome_of_interest = ["COPD", "SUBSTANCEMISUSE"]
outcome_token = dm.encode(outcome_of_interest)[0]
print(outcome_token)
# print(model(batch))

In [ ]:
Hs, labels = [], []
mins,maxes=[],[]
for _idx, batch in enumerate(dm.test_dataloader()):

    batch = replace_last_non_pad_with_pad(batch)
    print(batch["tokens".shape)
    outputs, _, hidden_states = model(batch, is_generation=True)
    print(outputs)
    
    hidden_states = hidden_states.cpu().detach().numpy()                           # (64, 128, 384) 
    Hs.append( hidden_states.reshape(hidden_states.shape[0], -1) )
    labels.append((batch["target_token"] == outcome_token).long().numpy())

    if _idx == 9:
        break



# Visualise hidden dimension labelled by target

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

H = np.concatenate(Hs, 0)
lbl = np.concatenate(labels, 0)

H = StandardScaler().fit_transform(H)
reducer = umap.UMAP()
H_proj = reducer.fit_transform(H)

plt.close()
plt.scatter(H_proj[:,0], H_proj[:,1], c=lbl)
plt.savefig(save_path + f"zero_shot/hidden_umap.png")

In [ ]:
print(outputs["surv"]["surv_CDF"][outcome_token].shape)

# The first two tokens in the vocab correspond to the PAD and UNK tokens. There is no CDF corresponding to the PAD token, so the indexing for surv_CDF begins as ["UNK", "ADDISONS_DISEASE", ...]
# print(dm.decode([0,1,2]))

outcomes = ["COPD", "SUBSTANCEMISUSE"]
outcome_tokens = dm.encode(outcomes)

# for outcome in outcomes:
    # observed_outcome_token = dm.encode([outcome])[0]
cdf = np.zeros_like(outputs["surv"]["surv_CDF"][0])
lbls = np.zeros_like(batch["target_token"])

for _outcome_token in outcome_tokens:
    cdf += outputs["surv"]["surv_CDF"][_outcome_token - 1] 
    lbls += (batch["target_token"] == _outcome_token).long().numpy()

plt.close()
cdf_true = cdf[lbls==1,:]
cdf_false = cdf[lbls==0,:]
for i in range(cdf_true.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_true[i,:], c="r", label="outcome occurred next" if i == 0 else None, alpha=1)
for i in range(cdf_false.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_false[i,:], c="k", label="outcome did not occur next" if i == 0 else None, alpha=0.3)

plt.legend(loc=2)
plt.xlabel("days")
plt.ylabel(f"P(t>T) - outcomes={','.join(outcomes)}")
plt.savefig(save_path + f"zero_shot/cdf_outcomes.png")

In [ ]:
print(batch["target_token"].unique())
print(len(outputs["surv"]["surv_CDF"]))

In [ ]:
dm.decode([2])

In [ ]:
outputs["surv"]["surv_CDF"][observed_outcome_token - 1]